# Distributed Tensorflow training using Horovod via OCI Jobs

## Contents

1. [Background](#Background)
1. [Prerequisites](#Prerequisites)
1. [Train](#Train)
1. [Setup IAM](#Setup%20IAM)
1. [Build](#Build)

---

## Background

Horovod is a distributed deep learning training framework for TensorFlow, Keras, PyTorch, and MXNet. This notebook example shows how to use Horovod with Tensorflow in OCI Data Science Jobs . OCI Data Science currently support Elastic Horovod workloads with gloo backend.

For more information about the Horovod with TensorFlow , please visit [Horovod-Tensorflow](https://horovod.readthedocs.io/en/stable/tensorflow.html)

---


## Prerequisites

### 1. Install ads package >= 2.5.9

In [ ]:
!pip3 install oracle-ads

### 2. Install docker:

https://docs.docker.com/get-docker

### 3. Set IAM Policies

Following Policies need to be in place in the OCI IAM service. This would allow OCI datascience job runs to access needed services such as logging, object storage, vcns etc.

#### Create the Dynamic Group
```
ALL {resource.type = ‘datasciencejobrun’, resource.compartment.id = <COMPARTMENT_OCID>}
```

#### Create policies
```
Allow dynamic-group <DYNAMIC_GROUP_NAME> to use log-content in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to use log-groups in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to inspect repos in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to inspect vcns in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to manage objects in compartment <COMPARTMENT_NAME> where any {target.bucket.name='<BUCKET_NAME>'}
Allow dynamic-group <DYNAMIC_GROUP_NAME> to manage buckets in compartment <COMPARTMENT_NAME> where any {target.bucket.name='<BUCKET_NAME>'}
```


### 4. Create VCN and private subnet

Horovod Distributed Training requires nodes to communicate to each other. Therefor, network settings need to be provisioned. Create a VCN and a private subnet. Ths subnet id of this private subnet needs to be configured in the workload yaml file ('train.yaml').

## Trainining Script

This section will create a horovod tensorflow training script. This is the training code that executes on the horovod cluster. The script must confirm to Elastic Horovod apis.

The following script uses Horovod framework for distributed training where Horovod related apis are commented starting with `Horovod:`. <br> For example, `Horovod: add Horovod DistributedOptimizer`, `Horovod: initialize optimize`, etc.

In [ ]:
%%writefile train.py

# Script adapted from https://github.com/horovod/horovod/blob/master/examples/elastic/tensorflow2/tensorflow2_keras_mnist_elastic.py

# ==============================================================================



import argparse
import tensorflow as tf
import horovod.tensorflow.keras as hvd
from distutils.version import LooseVersion

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

parser = argparse.ArgumentParser(description='Tensorflow 2.0 Keras MNIST Example')

parser.add_argument('--use-mixed-precision', action='store_true', default=False,
                    help='use mixed precision for training')

parser.add_argument('--data-dir',
                    help='location of the training dataset in the local filesystem (will be downloaded if needed)',
                    default='/code/data/mnist.npz')

args = parser.parse_args()

if args.use_mixed_precision:
    print(f"using mixed precision {args.use_mixed_precision}")
    if LooseVersion(tf.__version__) >= LooseVersion('2.4.0'):
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy('mixed_float16')
    else:
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
        tf.keras.mixed_precision.experimental.set_policy(policy)

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

import numpy as np
minist_local = args.data_dir

def load_data():
    print("using pre-fetched dataset")
    with np.load(minist_local, allow_pickle=True) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)

(mnist_images, mnist_labels), _ = load_data() if os.path.exists(minist_local) else tf.keras.datasets.mnist.load_data(path='mnist-%d.npz' % hvd.rank())


dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
             tf.cast(mnist_labels, tf.int64))
)
dataset = dataset.repeat().shuffle(10000).batch(128)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Horovod: adjust learning rate based on number of GPUs.
scaled_lr = 0.001 * hvd.size()
opt = tf.optimizers.Adam(scaled_lr)

# Horovod: add Horovod DistributedOptimizer.
opt = hvd.DistributedOptimizer(
    opt, backward_passes_per_step=1, average_aggregated_gradients=True)

# Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
# uses hvd.DistributedOptimizer() to compute gradients.
model.compile(loss=tf.losses.SparseCategoricalCrossentropy(),
                    optimizer=opt,
                    metrics=['accuracy'],
                    experimental_run_tf_function=False)

# Horovod: initialize optimizer state so we can synchronize across workers
# Keras has empty optimizer variables() for TF2:
# https://sourcegraph.com/github.com/tensorflow/tensorflow@v2.4.1/-/blob/tensorflow/python/keras/optimizer_v2/optimizer_v2.py#L351:10
model.fit(dataset, steps_per_epoch=1, epochs=1, callbacks=None)

state = hvd.elastic.KerasState(model, batch=0, epoch=0)

def on_state_reset():
    tf.keras.backend.set_value(state.model.optimizer.lr,  0.001 * hvd.size())
    # Re-initialize, to join with possible new ranks
    state.model.fit(dataset, steps_per_epoch=1, epochs=1, callbacks=None)

state.register_reset_callbacks([on_state_reset])

callbacks = [
    hvd.callbacks.MetricAverageCallback(),
    hvd.elastic.UpdateEpochStateCallback(state),
    hvd.elastic.UpdateBatchStateCallback(state),
    hvd.elastic.CommitStateCallback(state),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
# save the artifacts in the OCI__SYNC_DIR dir.
artifacts_dir=os.environ.get("OCI__SYNC_DIR") + "/artifacts"
tb_logs_path = os.path.join(artifacts_dir,"logs")
check_point_path =  os.path.join(artifacts_dir,"ckpts",'checkpoint-{epoch}.h5')
if hvd.rank() == 0:
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(check_point_path))
    callbacks.append(tf.keras.callbacks.TensorBoard(tb_logs_path))

# Train the model.
# Horovod: adjust number of steps based on number of GPUs.
@hvd.elastic.run
def train(state):
    state.model.fit(dataset, steps_per_epoch=500 // hvd.size(),
                    epochs=2-state.epoch, callbacks=callbacks,
                    verbose=1)

train(state)

## Build

### Initialize a distributed-training folder
Next step would be to create a distributed-training workspace. Execute the following command to fetch the 'horovod-tensorflow' framework. This would create a directory 'oci_dist_training_artifacts'. The directory essentially contains artifacts(dockerfile, configurations, gloo code etc) to build a horovod job docker image.

In [ ]:
!ads opctl distributed-training init --framework horovod-tensorflow --version v1

### Build Docker image

In [ ]:
IMAGE_NAME='hvdjob-cpu-tf'
IMAGE_TAG=1.0

In [ ]:
!docker build -f oci_dist_training_artifacts/horovod/v1/docker/tensorflow.cpu.Dockerfile -t $IMAGE_NAME:$IMAGE_TAG .

The training code('train.py') is assumed to be in the current working directory. This can be overwritten using the 'CODE_DIR' build arg.

`docker build --build-arg CODE_DIR=<code_folder> -f oci_dist_training_artifacts/horovod/docker/tensorflow.cpu.Dockerfile -t $IMAGE_NAME:$IMAGE_TAG .`


### Push the Docker Image to your Tenancy OCIR
Steps
1. Follow the instructions to setup container registry from [here](https://docs.oracle.com/en-us/iaas/Content/Registry/Tasks/registrypushingimagesusingthedockercli.htm)
2. Make sure you create a repository in OCIR to push the image
3. Tag Local Docker image that needs to be pushed. 
4. Push the Docker image from your machine to OCI Container Registry. 

#### Tag Docker image
Please replace the TENANCY_NAMESPACE (you can find this in tenancy information on oci console) and REGION_CODE [iad|phx ..]

In [ ]:
!docker tag $IMAGE_NAME:$IMAGE_TAG iad.ocir.io/<TENANCY_NAMESPACE>/horovod:$IMAGE_NAME:$IMAGE_TAG

# Example: !docker tag $IMAGE_NAME:$IMAGE_TAG iad.ocir.io/ociodscdev/horovod/$IMAGE_NAME:$IMAGE_TAG

#### Push Docker Image

In [ ]:
!docker push <REGION_CODE>.ocir.io/<TENANCY_NAMESPACE>/horovod/$IMAGE_NAME:$IMAGE_TAG

#Example: !docker push iad.ocir.io/ociodscdev/horovod/$IMAGE_NAME:$IMAGE_TAG

## Run

### Define your workload yaml:

The yaml file is a declarative way to express the workload.
Create a workload yaml file called `train.yaml` to specify the run config.

Workload yaml file has the following format.
<br>

```yaml
kind: distributed
apiVersion: v1.0
spec:
  infrastructure: # This section maps to Job definition. Does not include environment variables
    kind: infrastructure
    type: dataScienceJob
    apiVersion: v1.0
    spec:
      projectId: oci.xxxx.<project_ocid>
      compartmentId: oci.xxxx.<compartment_ocid>
      displayName: HVD-Distributed-TF
      logGroupId: oci.xxxx.<log_group_ocid>
      logId: oci.xxx.<log_ocid>
      subnetId: oci.xxxx.<subnet-ocid>
      shapeName: VM.Standard2.4
      blockStorageSize: 50
  cluster:
    kind: HOROVOD
    apiVersion: v1.0
    spec:
      image: "iad.ocir.io/<tenancy_id>/<repo_name>/<image_name>:<image_tag>"
      workDir:  "oci://<bucket_name>@<bucket_namespace>/<bucket_prefix>"
      name: "horovod_tf"
      config:
        env:
          # MIN_NP, MAX_NP and SLOTS are inferred from the shape. Modify only when needed.
          # - name: MIN_NP
          #   value: 2
          # - name: MAX_NP
          #   value: 4
          # - name: SLOTS
          #   value: 2
          - name: WORKER_PORT
            value: 12345
          - name: START_TIMEOUT #Optional: Defaults to 600.
            value: 600
          - name: ENABLE_TIMELINE # Optional: Disabled by Default.Significantly increases training duration if switched on (1).
            value: 0
          - name: SYNC_ARTIFACTS #Mandatory: Switched on by Default.
            value: 1
          - name: WORKSPACE #Mandatory if SYNC_ARTIFACTS==1: Destination object bucket to sync generated artifacts to.
            value: "<bucket_name>"
          - name: WORKSPACE_PREFIX #Mandatory if SYNC_ARTIFACTS==1: Destination object bucket folder to sync generated artifacts to.
            value: "<bucket_prefix>"
          - name: HOROVOD_ARGS # Parameters for cluster tuning.
            value: "--verbose"
      main:
        name: "scheduler"
        replicas: 1 #this will be always 1
      worker:
        name: "worker"
        replicas: 2 #number of workers
  runtime:
    kind: python
    apiVersion: v1.0
    spec:
      entryPoint: "/code/train.py" #location of user's training script in docker image.
      args:  #any arguments that the training script requires.
      env:
```
<br> <br>
The following variables are tenancy specific that needs to be modified.

| Variable | Description |
| :-------- | :----------- |
|compartmentId|OCID of the compartment where Data Science projects are created|
|projectId|OCID of the project created in Data Science service|
|subnetId|OCID of the subnet attached your Job|
|logGroupId|OCID of the log group for JobRun logs|
|image|Image from OCIR to be used for JobRuns|
|workDir|URL to the working directory for opctl|
|WORKSPACE|Workspace with the working directory to be used|
|entryPoint|The script to be executed when launching the container|

In [ ]:
%%writefile train.yaml

kind: distributed
apiVersion: v1.0
spec:
  infrastructure: # This section maps to Job definition. Does not include environment variables
    kind: infrastructure
    type: dataScienceJob
    apiVersion: v1.0
    spec:
      projectId: oci.xxxx.<project_ocid>
      compartmentId: oci.xxxx.<compartment_ocid>
      displayName: HVD-Distributed-TF
      logGroupId: oci.xxxx.<log_group_ocid>
      logId: oci.xxx.<log_ocid>
      subnetId: oci.xxxx.<subnet-ocid>
      shapeName: VM.Standard2.4
      blockStorageSize: 50
  cluster:
    kind: HOROVOD
    apiVersion: v1.0
    spec:
      image: "iad.ocir.io/<tenancy_id>/<repo_name>/<image_name>:<image_tag>"
      workDir:  "oci://<bucket_name>@<bucket_namespace>/<bucket_prefix>"
      name: "horovod_tf"
      config:
        env:
          # MIN_NP, MAX_NP and SLOTS are inferred from the shape. Modify only when needed.
          # - name: MIN_NP
          #   value: 2
          # - name: MAX_NP
          #   value: 4
          # - name: SLOTS
          #   value: 2
          - name: WORKER_PORT
            value: 12345
          - name: START_TIMEOUT #Optional: Defaults to 600.
            value: 600
          - name: ENABLE_TIMELINE # Optional: Disabled by Default.Significantly increases training duration if switched on (1).
            value: 0
          - name: SYNC_ARTIFACTS #Mandatory: Switched on by Default.
            value: 1
          - name: WORKSPACE #Mandatory if SYNC_ARTIFACTS==1: Destination object bucket to sync generated artifacts to.
            value: "<bucket_name>"
          - name: WORKSPACE_PREFIX #Mandatory if SYNC_ARTIFACTS==1: Destination object bucket folder to sync generated artifacts to.
            value: "<bucket_prefix>"
          - name: HOROVOD_ARGS # Parameters for cluster tuning.
            value: "--verbose"
      main:
        name: "scheduler"
        replicas: 1 #this will be always 1
      worker:
        name: "worker"
        replicas: 2 #number of workers
  runtime:
    kind: python
    apiVersion: v1.0
    spec:
      entryPoint: "/code/train.py" #location of user's training script in docker image.
      args:  #any arguments that the training script requires.
      env:

### Use ads opctl to create the cluster infrastructure and run the workload.

#### Dry Run To check the runtime configurations.

In [ ]:
! ads opctl run -f train.yaml --dry-run

#### Submit the workload

In [ ]:
! ads opctl run -f train.yaml

This would emit the following information about the job created and the job runs within.<br>
`jobId: <job_id>`<br>
`mainJobRunId: <scheduer_job_run_id>`<br>
`workDir: oci://<bucket_name>@<bucket_namespace>/<bucket_prefix>`<br>
`workerJobRunIds:`<br>
`- <worker_1_jobrun_id>`<br>
`- <worker_2_jobrun_id>`<br>

#### Monitor logs
You can monitor the logs emitted from the job runs using the following command.

In [ ]:
! ads jobs watch <jobrun_id>